In [1]:
import torch
from torch.autograd.function import InplaceFunction, Function
import torch.nn as nn
import torch.nn.functional as F
import math

In [9]:
Input = torch.Tensor([2.1])

In [10]:
Input.clamp_(0, 5).round_()

tensor([2.])

In [12]:
a = torch.rand(20,10,3)

In [18]:
a.view(a.size(0),-1).min(-1)

(tensor([0.1143, 0.0202, 0.0626, 0.0347, 0.0578, 0.0032, 0.0907, 0.0410, 0.0370,
         0.0522, 0.0041, 0.1042, 0.0102, 0.0070, 0.0128, 0.0366, 0.0298, 0.0051,
         0.0244, 0.0171]),
 tensor([13, 14,  3,  8,  6,  2,  8, 20,  8,  6, 17, 17, 19,  6, 15, 19, 26, 25,
         29, 18]))

In [19]:
a.shape

torch.Size([20, 10, 3])

In [23]:
y = a.view(a.shape[0]//20,-1)

In [25]:
y.size()

torch.Size([1, 600])

In [28]:
y.min(-1)[0].mean(-1)

tensor(0.0032)

In [29]:
conv_layer = nn.Conv2d(3, 5, kernel_size=3, stride=1,
                     padding=1, bias=False)

In [30]:
conv_layer.weight.size()

torch.Size([5, 3, 3, 3])

In [32]:
conv_layer.weight[0,:,:,:].size()

torch.Size([3, 3, 3])

In [33]:
class UniformQuantize(InplaceFunction):

    @classmethod
    def forward(cls, ctx, input, num_bits=8, min_value=None, max_value=None,
                stochastic=False, inplace=False, enforce_true_zero=False, num_chunks=None, out_half=False):

        num_chunks = num_chunks = input.shape[
            0] if num_chunks is None else num_chunks
        if min_value is None or max_value is None:
            B = input.shape[0]
            y = input.view(B // num_chunks, -1)
        if min_value is None:
            min_value = y.min(-1)[0].mean(-1)  # C
            #min_value = float(input.view(input.size(0), -1).min(-1)[0].mean())
        if max_value is None:
            #max_value = float(input.view(input.size(0), -1).max(-1)[0].mean())
            max_value = y.max(-1)[0].mean(-1)  # C
        ctx.inplace = inplace
        ctx.num_bits = num_bits
        ctx.min_value = min_value
        ctx.max_value = max_value
        ctx.stochastic = stochastic

        if ctx.inplace:
            ctx.mark_dirty(input)
            output = input
        else:
            output = input.clone()

        qmin = 0.
        qmax = 2.**num_bits - 1.
        #import pdb; pdb.set_trace()
        scale = (max_value - min_value) / (qmax - qmin)

        scale = max(scale, 1e-8)

        if enforce_true_zero:
            initial_zero_point = qmin - min_value / scale
            zero_point = 0.
            # make zero exactly represented
            if initial_zero_point < qmin:
                zero_point = qmin
            elif initial_zero_point > qmax:
                zero_point = qmax
            else:
                zero_point = initial_zero_point
            zero_point = int(zero_point)
            output.div_(scale).add_(zero_point)
        else:
            output.add_(-min_value).div_(scale).add_(qmin)

        if ctx.stochastic:
            noise = output.new(output.shape).uniform_(-0.5, 0.5)
            output.add_(noise)
        output.clamp_(qmin, qmax).round_()  # quantize

        if enforce_true_zero:
            output.add_(-zero_point).mul_(scale)  # dequantize
        else:
            output.add_(-qmin).mul_(scale).add_(min_value)  # dequantize
        if out_half and num_bits <= 16:
            output = output.half()
        return output

    @staticmethod
    def backward(ctx, grad_output):
        # straight-through estimator
        grad_input = grad_output
        return grad_input, None, None, None, None, None, None


In [34]:
a = torch.rand(4,5)

In [36]:
from torch.autograd import Variable

In [37]:
a = Variable(a)

In [40]:
b = UniformQuantize().apply(a,8,None,None,False)

In [41]:
a

tensor([[0.1503, 0.8787, 0.4591, 0.2252, 0.3175],
        [0.3072, 0.3202, 0.7554, 0.4125, 0.8316],
        [0.9673, 0.9366, 0.4022, 0.6118, 0.6386],
        [0.4410, 0.4244, 0.5805, 0.5696, 0.7735]])

In [42]:
b.size()

torch.Size([4, 5])

In [45]:
s = conv_layer.weight[0,:,:,:]

In [46]:
w = conv_layer.weight[1,:,:,:]

In [51]:
torch.cat([w.unsqueeze(0)]).size()

torch.Size([1, 3, 3, 3])